# CrossTorus experiments

In this notebook, we study matching diagrams across Navground simulations on the cross torus. In particular, we compare four different navigation behaviours: ORCA, HRVO, HL, SocialForce and Dummy

First, let us import a few important modules for this task.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import os

from navground import core, sim
from navground.sim.ui.video import display_video_from_run

import perdiver.perdiver as perdiver
from perdiver.navground_io import parser, run_navground
from perdiver.distances import *

plots_dir = os.path.join("plots", "matchings")
experiment_dir = "experiments"
os.makedirs(plots_dir, exist_ok=True)
os.makedirs(experiment_dir, exist_ok=True)

Next, let us execute the Navground cross torus experiment. We run the experiment 15 times.

In [ ]:
args = parser.parse_args([
        '--scenario', 'CrossTorus',
        '--side', '6.5',
        '--num_runs', '15',
        '--num_steps', '500',
        '--time_step', '0.1',
        '--num_agents', '12',
        '--max_speed', '1.66',
        '--optimal_speed_min', '0.1',
        '--optimal_speed_min', '0.15',
        '--radius', '0.4',
        '--safety_margin', '0.1',
        '--epsilon', '20',
        '--time_delay', '5',
])
behavior_list = ["ORCA", "HL", "HRVO", "SocialForce"]
runs = {}
for behavior in behavior_list:
    args.behavior = behavior
    runs[behavior] = run_navground(args)

First, let us visualise the HL experiment.

In [ ]:
display_video_from_run(run=runs["HL"][0], factor=6.0, fps=20)

Now, we visualise the ORCA experiment.

In [ ]:
display_video_from_run(run=runs["ORCA"][0], factor=6.0, fps=20)

Also HRVO and Dummy

In [ ]:
display_video_from_run(run=runs["HRVO"][0], factor=6.0, fps=20)

In [ ]:
display_video_from_run(run=runs["SocialForce"][0], factor=6.0, fps=20)

Both simulations are very different. In partiuclar, we observe mainly two dynamics, either the robots end up going in straight trajectories or they get stuck. Both examples below where produced with the same variables.

### Pairwise Matchings

We are now going to compute the induced matchings and their associated diagrams.

Also, we set up the variable "weight" and the timestep shift for our experiments.

Now, we are going to start by considering two timesteps and their associated divergence diagrams. Notice that these do not change much.

In [ ]:
from perdiver.distances import  distances_2Dtorus_weighted_velocities
from perdiver.perdiver import get_matching_diagram
from perdiver.perdiver import plot_timesteps_cross_torus, plot_matching_diagram, same_diagram_scale

args.weight = 1
args.start_step = 20
gs_kw = dict(width_ratios=[1,1], height_ratios=[1,1])
sb = ["HL", "ORCA"]
fig, axd = plt.subplot_mosaic([[f'points_{sb[0]}', f'Diag_{sb[0]}'],
                               [f'points_{sb[1]}', f'Diag_{sb[1]}']],
                              gridspec_kw=gs_kw, figsize=(8, 8),
                              layout="constrained")
for behavior in sb:
    run = runs[behavior][0]
    ps = np.array(run.poses)
    twists = np.array(run.twists)
    X = ps[args.start_step]
    Y = ps[args.start_step + args.epsilon]
    vel_X = twists[args.start_step]
    vel_Y = twists[args.start_step + args.epsilon]
    X_len = X.shape[0]-1
    # Plot two timesteps
    ax = axd[f"points_{behavior}"]
    plot_timesteps_cross_torus(run, [args.start_step, args.start_step + args.epsilon], args.side, ax) 
    ax.set_title(f"points_{behavior}", fontsize=20)
    # Plot matching diagram
    ax = axd[f"Diag_{behavior}"]
    Dist_X = distances_2Dtorus_weighted_velocities(X, vel_X, args.weight, args.side)
    Dist_Y = distances_2Dtorus_weighted_velocities(Y, vel_Y, args.weight, args.side)
    match_diagram = get_matching_diagram(Dist_X, Dist_Y)
    plot_matching_diagram(match_diagram, ax, color="blue")
    ax.set_title(behavior, fontsize=20)
# end for
same_diagram_scale([axd[f'Diag_{sb[0]}'], axd[f'Diag_{sb[1]}']])
plt.savefig(os.path.join(plots_dir, f"two_timesteps_cross_torus_{sb[0]}_{sb[1]}.png"))

Last, we compute the persistence matching diagrams across behaviours, runs and step list. 

In [ ]:
import scipy.spatial.distance as dist

args.steps_list = list(range(0, args.num_steps-args.epsilon-args.time_delay, args.time_delay))
args.weight = 2

diagrams_behavior = {}
for j, behavior in enumerate(behavior_list):
    diagrams_run_list = []
    for i_run in range(args.num_runs):
        ps = np.array(runs[behavior][i_run].poses)
        twists = np.array(runs[behavior][i_run].twists)
        diagrams_list = []
        for idx, start_step in enumerate(args.steps_list):
            if ps.shape[0] > start_step + args.epsilon:
                X = ps[start_step]
                Y = ps[start_step + args.epsilon]
                vel_X = twists[start_step]
                vel_Y = twists[start_step + args.epsilon]
                Dist_X = distances_2Dtorus_weighted_velocities(X, vel_X, args.weight, args.side)
                Dist_Y = distances_2Dtorus_weighted_velocities(Y, vel_Y, args.weight, args.side)
                match_diagram = perdiver.get_matching_diagram(Dist_X, Dist_Y)
                diagrams_list.append(match_diagram)
            else: # Add matching diagram from previus loop
                diagrams_list.append(match_diagram)

        # end for over start steps
        diagrams_run_list.append(np.array(diagrams_list))
    # end for over runs
    diagrams_behavior[behavior] = diagrams_run_list
# end for over behaviors

Next, we plot the evolution of such diagrams for the behaviours, on the first run.

In [ ]:
fig, ax = plt.subplots(ncols=len(behavior_list), figsize=(5*len(behavior_list),4))
for j, behavior in enumerate(behavior_list):
    diagrams_run = diagrams_behavior[behavior][0] # Take first run only
    for idx, start_step in enumerate(args.steps_list):
        perdiver.plot_matching_diagram(diagrams_run[idx], ax[j], color=mpl.colormaps["GnBu"](idx/len(args.steps_list)))
    # end for 
    norm = mpl.colors.Normalize(vmin=args.steps_list[0], vmax=args.steps_list[-1])
    cmap = mpl.colormaps["GnBu"]
    mappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
    plt.colorbar(mappable=mappable, ax=ax[j])
    ax[j].set_title(f"Evolution matching diagram {behavior}")
# end for
perdiver.same_diagram_scale([ax[i] for i in range(4)])
plt.savefig(os.path.join(plots_dir, f"evolution_matching_behaviors.png"))

## Vectorisations of matching diagrams

### Vectorisation 1: timestep means of persistence images

We vectorise the persistence diagrams using persistence images. In particular, for each run, we obtain a single image given by taking the mean across the simulation.

In [ ]:
from gudhi import representations

npixels = 15
perim = representations.PersistenceImage(resolution=[npixels, npixels], bandwidth=0.1, im_range=[0,3,-3,3])
### Put together all persistence diagrams for fitting persistence images
all_diagrams = []
for behavior in behavior_list:
    for i_run in range(args.num_runs): 
        for matching_diagram in diagrams_behavior[behavior][i_run]:
            all_diagrams.append(matching_diagram)
        # for over matching diagrams from run
    # for over runs
# for over behaviors
perim.fit(all_diagrams)
### Compute persistence images for all behaviors and runs
perim_arr_dict = {}
for behavior in behavior_list:
    perim_means_list = []
    for i_run in range(args.num_runs): 
        perim_means_list.append(perim.transform(diagrams_behavior[behavior][i_run]))
    # end for
    perim_arr_dict[behavior] = perim_means_list
# end for
### Compute means across timesteps of persistence images for all behaviors and runs
perim_means = {}
for behavior in behavior_list:
    perim_means_list = []
    for i_run in range(args.num_runs): 
        perim_means_list.append(perim_arr_dict[behavior][i_run].mean(axis=0).reshape(15,15))
    # end for over runs
    perim_means[behavior] = perim_means_list
# end for over behaviors

Visualise persistence images.

In [ ]:
fig, ax = plt.subplots(ncols=len(behavior_list), figsize=(3.5*len(behavior_list),3), layout="constrained")
for idx, behavior in enumerate(behavior_list):
    perim = perim_means[behavior][0] # only take the first run
    ax[idx].imshow(perim)
    ax[idx].set_title(behavior, fontsize=20)
# end for 
# plt.tight_layout()
plt.savefig(os.path.join(plots_dir, f"persistence_images_means.png"))

Use PCA to detect clusters of persistence images.

In [ ]:
color_behavior = {}
for i, behavior in enumerate(behavior_list):
    color_behavior[behavior] = mpl.colormaps["Set1"](i / (len(behavior_list) +1))
# end for
marker_behavior = {"ORCA": "o", "HL": "X", "HRVO": "+", "Dummy": "*", "SocialForce": "x"}

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import minmax_scale

# Start by ransforming all persistence images into vectors
all_perim_transformed = []
for i, behavior in enumerate(behavior_list):
    for image in perim_means[behavior]:
        image_transformed = image.reshape((npixels,npixels))
        all_perim_transformed.append(image_transformed.ravel())
    # end for over runs
# for over behaviors

# Create and fit PCA to image persistence means, transforming to 2 d
pca = PCA(n_components=2)
Y = pca.fit_transform(all_perim_transformed)
# Separate projections by behavior
Y_dict = {}
for i, behavior in enumerate(behavior_list):
    Y_dict[behavior] = Y[args.num_runs*i:args.num_runs*(i+1)]
# end for
# Plot PCA projections
fig, ax = plt.subplots(figsize=(5,5))
for behavior in behavior_list:
    ax.scatter(Y_dict[behavior][:,0], Y_dict[behavior][:,1], color=color_behavior[behavior], label=behavior, marker=marker_behavior[behavior])

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.1,0),  ncol=len(behavior_list))
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "PCA-persim-means.png"))

### Vectorisation 2: persistence divergence images (perdiver images)

Next, we compute and print the divergence array across a few simulation steps. Basically, a persistence divergence image is an array where the horizontal axes goes along timesteps, while the vertical axes has the size the total number of agents. For each column in the array, we include the divergence scores sorted, from lower to higher.

To start, we compute divergence vectors from the matching diagrams.

In [ ]:
from perdiver.perdiver import compute_divergence_vector

divergence_dict = {}
for behavior in behavior_list:
    divergence_runs = []
    for i_run in range(args.num_runs):
        divergence_list = [] 
        for matching_diagram in diagrams_behavior[behavior][i_run]:
            divergence_vector = np.sort(compute_divergence_vector(matching_diagram))
            divergence_list.append(divergence_vector)
        # for over matching diagrams from run
        divergence_runs.append(np.array(divergence_list).transpose())
    # for over runs
    divergence_dict[behavior] = divergence_runs
# for over behaviors

Next, we plot some of the divergence images for illustration.

In [ ]:
vmax = np.max([np.max(divergence_dict[behavior]) for behavior in divergence_dict.keys()])
vmin = np.min([np.min(divergence_dict[behavior]) for behavior in divergence_dict.keys()])
for behavior in divergence_dict.keys():
    divergence_arr = divergence_dict[behavior][0]
    ## Save figure 
    fig, ax = plt.subplots(figsize=(6,1.4))
    mapable = ax.imshow(divergence_arr, aspect="auto", vmax=vmax, vmin=vmin, extent=(args.steps_list[0], args.steps_list[-1], 0, X.shape[0]))
    ax.set_title(f"Divergence {behavior}")
    plt.colorbar(mapable)
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, f"perdiver_images_{behavior}.png"))

In [ ]:
# Start by ransforming all perdiver images into vectors
perdiver_im_transformed = []
for i, behavior in enumerate(behavior_list):
    for image in divergence_dict[behavior]:
        perdiver_im_transformed.append(image.ravel())
    # end for over runs
# for over behaviors

# Create and fit PCA to image persistence means, transforming to 2 d
pca = PCA(n_components=2)
Y = pca.fit_transform(perdiver_im_transformed)
# Separate projections by behavior
Y_dict = {}
for i, behavior in enumerate(behavior_list):
    Y_dict[behavior] = Y[args.num_runs*i:args.num_runs*(i+1)]
# end for
color_behavior = {}
for i, behavior in enumerate(behavior_list):
    color_behavior[behavior] = mpl.colormaps["Set1"](i / (len(behavior_list) +1))
# Plot PCA projections
fig, ax = plt.subplots(figsize=(5,5))
for behavior in behavior_list:
    ax.scatter(Y_dict[behavior][:,0], Y_dict[behavior][:,1], color=color_behavior[behavior], label=behavior, marker=marker_behavior[behavior])

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.1,0),  ncol=len(behavior_list))
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "PCA-perdiver-images.png"))

### Vectorisation 3: (absolute) persistence divergence

We perform a lineplot for each divergence.

In [ ]:
from perdiver.perdiver import absolute_perdiver_signal

absdiver_dict = {}
for behavior in behavior_list:
    absdiver_runs = []
    for i_run in range(args.num_runs):
        absdiver_runs.append(
            absolute_perdiver_signal(diagrams_behavior[behavior][i_run])
        )
    # for over runs
    absdiver_dict[behavior] = absdiver_runs
# for over behaviors

Plot absdiver signals

In [ ]:
quartile_absdiver = {}
for behavior in behavior_list:
    absdiver_runs = np.vstack(absdiver_dict[behavior])
    quartile_absdiver[behavior] = np.percentile(absdiver_runs, [25, 50, 75], axis=0)
# end for

In [ ]:
behavior_list

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
for behavior in behavior_list:
    quartile = quartile_absdiver[behavior]
    ax.plot(args.steps_list, quartile[1], color=color_behavior[behavior], label=behavior)
    ax.fill_between(args.steps_list, quartile[0], quartile[2], color=color_behavior[behavior], alpha=.3)
# end plotting
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.1,0),  ncol=len(behavior_list))
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "absolute-persistence-divergence.png"))

In [ ]:
# Create and fit PCA to image persistence means, transforming to 2 d
pca = PCA(n_components=2)

absdiver_all = []
for behavior in behavior_list:
    absdiver_all += absdiver_dict[behavior]
# absdiver list 

Y = pca.fit_transform(absdiver_all)
# Separate projections by behavior
Y_dict = {}
for i, behavior in enumerate(behavior_list):
    Y_dict[behavior] = Y[args.num_runs*i:args.num_runs*(i+1)]
# end for
color_behavior = {}
for i, behavior in enumerate(behavior_list):
    color_behavior[behavior] = mpl.colormaps["Set1"](i / (len(behavior_list) +1))
# Plot PCA projections
fig, ax = plt.subplots(figsize=(5,5))
for behavior in behavior_list:
    ax.scatter(Y_dict[behavior][:,0], Y_dict[behavior][:,1], color=color_behavior[behavior], label=behavior, marker=marker_behavior[behavior])

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.1,0),  ncol=len(behavior_list))
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "PCA-perdiver-images.png"))